In [1]:
import pandas as pd
import numpy as np
import scipy.stats as sp
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mtp
mtp.rcParams["pdf.fonttype"] = 42

import warnings
warnings.filterwarnings('ignore')

In [2]:
tcgaDf = pd.read_csv("C:/Data/Lab/Carter_Lab/aneuploidy_data/TCGA_TP_allgene.csv")
hnsc = tcgaDf[tcgaDf.disease == "HNSC"]
neoantigen_data = pd.read_csv("C:/Data/Lab/Carter_Lab/neoantigen/neoantigen_burden.weak_strong_thresholds.req_min5_RNA_reads.tsv",sep = "\t")
neoantigen_data["patient_id"] = [ x.split("-")[2] for x in neoantigen_data["TCGA_Barcode"]]

hpv_neg_ids_cbioportal = pd.read_csv("C:/Data/Lab/Zanetti_Lab/HNSC/hnsc_tcga_pan_can_atlas_2018_clinical_data.tsv",sep = "\t")
hpv_pos_ids_cbioportal = [ x.split("-")[2] for x in hpv_neg_ids_cbioportal[hpv_neg_ids_cbioportal["Subtype"] == "HNSC_HPV+"]["Patient ID"].tolist()]

cd4 = ["CD3D","CD3E","CD3G","CD4"]
cd8 = ["CD3D","CD3E","CD3G","CD8A","CD8B"]
bcell = ["CD19","MS4A1"]
cd3 = ["CD3D","CD3E","CD3G"]

df = hnsc[list(set(cd3 + bcell + cd4 + cd8)) + ["TERT","patient_id"]]
df["cd3"] = sp.stats.gmean(np.log2(1+df[cd3]),axis=1)
df["bcell"] = sp.stats.gmean(np.log2(1+df[bcell]),axis=1)
df["cd4"] = sp.stats.gmean(np.log2(1+df[cd4]),axis=1)
df["cd8"] = sp.stats.gmean(np.log2(1+df[cd8]),axis=1)
df["TERT"] = np.log2(1+df["TERT"])

f_dir = "C:/Data/Lab/Zanetti_Lab/HNSC/"

In [3]:
def cox_p_analysis_tert_low(df,include_stage = False,plot = False, save = False):
    
    from lifelines import CoxPHFitter
    cph = CoxPHFitter()

    """@DIR is a global variable"""
    f_dir = "C:/Data/Lab/Zanetti_Lab/HNSC/"
    pfs = pd.read_csv(f_dir + "PFSfile.csv")
    clinic = pd.read_csv(f_dir + "clinical.csv")
    clinic.rename(columns = {"Unnamed: 0":"patient_id"},inplace = True)
    clinic["patient_id"] = [ x.split("-")[2] for x in clinic["patient_id"]]
    
    # get log Tumor mutation burden
    allmaf = pd.read_csv(f_dir + "all.maf",header = None, sep = "\t")
    tmb = pd.DataFrame(allmaf[0].value_counts())
    tmb.index = [ x.split("-")[2] for x in tmb.index.tolist()]
    tmb.reset_index(inplace = True)
    tmb.columns = ["patient_id","mutCount"]
    tmb["logTMB"] = np.log(1 + tmb["mutCount"])
    
    df = pd.merge(df,pfs[["patient_id","PFS.time","PFS"]], on = "patient_id")
    df = pd.merge(df,clinic[["patient_id","Sex","Age"]], on = "patient_id")
    df = pd.merge(df,tmb[["patient_id","logTMB"]], on = "patient_id")
    df["Sex"] = [1 if x == "Male" else 0 for x in df.Sex.tolist()]
    df.dropna(inplace = True)
    df.drop_duplicates(inplace = True)
    df.dropna()

    hpv_neg_ids_cbioportal = pd.read_csv("C:/Data/Lab/Zanetti_Lab/HNSC/hnsc_tcga_pan_can_atlas_2018_clinical_data.tsv",sep = "\t")
    hpv_pos_ids_cbioportal = [ x.split("-")[2] for x in hpv_neg_ids_cbioportal[hpv_neg_ids_cbioportal["Subtype"] == "HNSC_HPV+"]["Patient ID"].tolist()]
    
    df["hpv"] = [1 if x in hpv_pos_ids_cbioportal else 0 for x in df["patient_id"].tolist()]
    low_quantile = np.quantile(df["TERT"].tolist(),q = .3)
    df = df[df["TERT"] <= low_quantile]
    #print(df.shape)
    
    # if include stage:

    if include_stage:
        stage_dict = {"Stage I":1,"Stage II":2, "Stage III":3, "Stage IV":4}
        stage = pd.read_csv(f_dir + "stageinfo.csv",header = None)
        stage.columns = ["patient_id","stage"]
        df = pd.merge(df, stage, on = "patient_id").drop_duplicates()
        df["stage"] = [stage_dict[x] if x in stage_dict.keys() else np.nan for x in df["stage"].tolist()]
        cph.fit(df[["PFS.time","bcell","hpv","PFS","Age","Sex","logTMB","stage"]].dropna(), 'PFS.time', event_col='PFS')
        cph.print_summary()
    else:
        cph.fit(df[["logTMB","PFS.time","PFS","Age","Sex"]], 'PFS.time', event_col='PFS')
        cph.print_summary()
        
    if plot:
        save_dir = "C:/Data/Lab/Zanetti_Lab/Figures/"
        cph.plot()
        if save:
            if include_stage:
                plt.savefig(save_dir+".cph.stage.plot.pdf", bbox_inches = "tight")
            else:
                plt.savefig(save_dir+".cph.plot.pdf", bbox_inches = "tight")

cox_p_analysis_tert_low(df,include_stage = True)

<lifelines.CoxPHFitter: fitted with 122 total observations, 52 right-censored observations>
             duration col = 'PFS.time'
                event col = 'PFS'
      baseline estimation = breslow
   number of observations = 122
number of events observed = 70
   partial log-likelihood = -281.23
         time fit was run = 2023-02-03 22:57:26 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
bcell       0.08       1.09       0.19            -0.29             0.46                 0.75                 1.58
hpv        -0.15       0.86       0.47            -1.07             0.77                 0.34                 2.16
Age        -0.00       1.00       0.01            -0.03             0.02                 0.97                 1.02
Sex         0.09       1.10       0.32            -0.53             0.71                 0.59                 2.04
logTMB      0.06       1.07       0.17            -0.26             0.39                 0.77                 1.47
stage       0.21       1.24       0.15            -0.07             0.50                 0.93                 1.65

              z    p   -log2(p)
covariate                      
bcell      0.44 0.66       0.60
hpv       -0.32 0.75       0.42
Age       -0.16 0.88       0.19
Sex        0.29 0.77       0.37
logTMB     0.38 0.70       0.51
stage      1.48 0.14       2.84
---
Concordance = 0.54
Partial AIC = 574.46
log-likelihood ratio test = 3.79 on 6 df
-log2(p) of ll-ratio test = 0.50

In [4]:
def cox_p_analysis_tert_high(df,include_stage = False,plot = False, save = False):
    
    from lifelines import CoxPHFitter
    cph = CoxPHFitter()

    """@DIR is a global variable"""
    f_dir = "C:/Data/Lab/Zanetti_Lab/HNSC/"
    pfs = pd.read_csv(f_dir + "PFSfile.csv")
    clinic = pd.read_csv(f_dir + "clinical.csv")
    clinic.rename(columns = {"Unnamed: 0":"patient_id"},inplace = True)
    clinic["patient_id"] = [ x.split("-")[2] for x in clinic["patient_id"]]
    
    # get log Tumor mutation burden
    allmaf = pd.read_csv(f_dir + "all.maf",header = None, sep = "\t")
    tmb = pd.DataFrame(allmaf[0].value_counts())
    tmb.index = [ x.split("-")[2] for x in tmb.index.tolist()]
    tmb.reset_index(inplace = True)
    tmb.columns = ["patient_id","mutCount"]
    tmb["logTMB"] = np.log(1 + tmb["mutCount"])
    
    df = pd.merge(df,pfs[["patient_id","PFS.time","PFS"]], on = "patient_id")
    df = pd.merge(df,clinic[["patient_id","Sex","Age"]], on = "patient_id")
    df = pd.merge(df,tmb[["patient_id","logTMB"]], on = "patient_id")
    df["Sex"] = [1 if x == "Male" else 0 for x in df.Sex.tolist()]
    df.dropna(inplace = True)
    df.drop_duplicates(inplace = True)
    df.dropna()
    
    hpv_neg_ids_cbioportal = pd.read_csv("C:/Data/Lab/Zanetti_Lab/HNSC/hnsc_tcga_pan_can_atlas_2018_clinical_data.tsv",sep = "\t")
    hpv_pos_ids_cbioportal = [ x.split("-")[2] for x in hpv_neg_ids_cbioportal[hpv_neg_ids_cbioportal["Subtype"] == "HNSC_HPV+"]["Patient ID"].tolist()]
    
    df["hpv"] = [1 if x in hpv_pos_ids_cbioportal else 0 for x in df["patient_id"].tolist()]
    high_quantile = np.quantile(df["TERT"].tolist(),q = .7)
    df = df[df["TERT"] >= high_quantile].drop_duplicates()
    #print(df.shape)
    
    # if include stage:

    if include_stage:
        stage_dict = {"Stage I":1,"Stage II":2, "Stage III":3, "Stage IV":4}
        stage = pd.read_csv(f_dir + "stageinfo.csv",header = None)
        stage.columns = ["patient_id","stage"]
        df = pd.merge(df, stage, on = "patient_id").drop_duplicates()
        df["stage"] = [stage_dict[x] if x in stage_dict.keys() else np.nan for x in df["stage"].tolist()]
        cph.fit(df[["PFS.time","bcell","hpv","PFS","Age","Sex","logTMB","stage"]].dropna(), 'PFS.time', event_col='PFS')
        cph.print_summary()
    else:
        cph.fit(df[["logTMB","PFS.time","PFS","Age","Sex"]], 'PFS.time', event_col='PFS')
        cph.print_summary()
        
    if plot:
        save_dir = "C:/Data/Lab/Zanetti_Lab/Figures/"
        cph.plot()
        if save:
            if include_stage:
                plt.savefig(save_dir+".cph.stage.plot.pdf", bbox_inches = "tight")
            else:
                plt.savefig(save_dir+".cph.plot.pdf", bbox_inches = "tight")

cox_p_analysis_tert_high(df,include_stage = True)

<lifelines.CoxPHFitter: fitted with 108 total observations, 63 right-censored observations>
             duration col = 'PFS.time'
                event col = 'PFS'
      baseline estimation = breslow
   number of observations = 108
number of events observed = 45
   partial log-likelihood = -174.66
         time fit was run = 2023-02-03 22:57:34 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
bcell      -0.40       0.67       0.19            -0.77            -0.03                 0.46                 0.97
hpv        -0.24       0.78       0.46            -1.15             0.67                 0.32                 1.95
Age         0.02       1.02       0.01            -0.01             0.04                 0.99                 1.05
Sex        -0.20       0.81       0.34            -0.86             0.45                 0.42                 1.57
logTMB     -0.08       0.93       0.24            -0.54             0.39                 0.58                 1.48
stage       0.28       1.32       0.17            -0.06             0.61                 0.94                 1.85

              z    p   -log2(p)
covariate                      
bcell     -2.11 0.04       4.84
hpv       -0.52 0.60       0.73
Age        1.13 0.26       1.95
Sex       -0.61 0.54       0.88
logTMB    -0.32 0.75       0.41
stage      1.63 0.10       3.26
---
Concordance = 0.65
Partial AIC = 361.31
log-likelihood ratio test = 14.14 on 6 df
-log2(p) of ll-ratio test = 5.15

In [5]:
def cox_p_analysis_tert_low_neoantigen(df,antigen,include_stage = False,plot = False, save = False):
    
    from lifelines import CoxPHFitter
    cph = CoxPHFitter()

    """@DIR is a global variable"""
    f_dir = "C:/Data/Lab/Zanetti_Lab/HNSC/"
    pfs = pd.read_csv(f_dir + "PFSfile.csv")
    clinic = pd.read_csv(f_dir + "clinical.csv")
    clinic.rename(columns = {"Unnamed: 0":"patient_id"},inplace = True)
    clinic["patient_id"] = [ x.split("-")[2] for x in clinic["patient_id"]]
    
    # get log Tumor mutation burden
    allmaf = pd.read_csv(f_dir + "all.maf",header = None, sep = "\t")
    tmb = pd.DataFrame(allmaf[0].value_counts())
    tmb.index = [ x.split("-")[2] for x in tmb.index.tolist()]
    tmb.reset_index(inplace = True)
    tmb.columns = ["patient_id","mutCount"]
    tmb["logTMB"] = np.log(1 + tmb["mutCount"])
    
    df = pd.merge(df,pfs[["patient_id","PFS.time","PFS"]], on = "patient_id")
    df = pd.merge(df,clinic[["patient_id","Sex","Age"]], on = "patient_id")
    df = pd.merge(df,tmb[["patient_id","logTMB"]], on = "patient_id")
    df["Sex"] = [1 if x == "Male" else 0 for x in df.Sex.tolist()]
    df.dropna(inplace = True)
    df.drop_duplicates(inplace = True)
    df.dropna()
    
    df = pd.merge(df,neoantigen_data,on = "patient_id")
    df["hpv"] = [1 if x in hpv_pos_ids_cbioportal else 0 for x in df["patient_id"].tolist()]
    low_quantile = np.quantile(df["TERT"].tolist(),q = .3)
    df = df[df["TERT"] <= low_quantile].drop_duplicates()
    print(df.shape)
    
    # if include stage:

    if include_stage:
        stage_dict = {"Stage I":1,"Stage II":2, "Stage III":3, "Stage IV":4}
        stage = pd.read_csv(f_dir + "stageinfo.csv",header = None)
        stage.columns = ["patient_id","stage"]
        df = pd.merge(df, stage, on = "patient_id").drop_duplicates()
        df["stage"] = [stage_dict[x] if x in stage_dict.keys() else np.nan for x in df["stage"].tolist()]
        cph.fit(df[["PFS.time","PFS","Age","bcell","hpv","Sex",antigen,"stage"]].dropna(), 'PFS.time', event_col='PFS') # remove b cell
        cph.print_summary()
    else:
        cph.fit(df[[antigen,"PFS.time","PFS","Age","Sex",]], 'PFS.time', event_col='PFS')
        cph.print_summary()
        
    if plot:
        save_dir = "C:/Data/Lab/Zanetti_Lab/Figures/"
        cph.plot()
        if save:
            if include_stage:
                plt.savefig(save_dir+".cph.stage.plot.pdf", bbox_inches = "tight")
            else:
                plt.savefig(save_dir+".cph.plot.pdf", bbox_inches = "tight")

for antigen in ["I_thresh2","II_thresh20","I_thresh0.5","II_thresh10"]:
    cox_p_analysis_tert_low_neoantigen(df,antigen,include_stage = True)

(146, 25)


<lifelines.CoxPHFitter: fitted with 120 total observations, 50 right-censored observations>
             duration col = 'PFS.time'
                event col = 'PFS'
      baseline estimation = breslow
   number of observations = 120
number of events observed = 70
   partial log-likelihood = -279.51
         time fit was run = 2023-02-03 22:57:43 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
Age        -0.00       1.00       0.01            -0.03             0.02                 0.97                 1.02
bcell       0.08       1.08       0.20            -0.32             0.47                 0.73                 1.60
hpv        -0.19       0.83       0.47            -1.11             0.73                 0.33                 2.08
Sex         0.06       1.06       0.31            -0.55             0.66                 0.58                 1.93
I_thresh2  -0.00       1.00       0.01            -0.01             0.01                 0.99                 1.01
stage       0.19       1.21       0.15            -0.09             0.48                 0.91                 1.61

              z    p   -log2(p)
covariate                      
Age       -0.24 0.81       0.31
bcell      0.38 0.70       0.51
hpv       -0.40 0.69       0.53
Sex        0.18 0.86       0.22
I_thresh2 -0.11 0.91       0.14
stage      1.33 0.18       2.46
---
Concordance = 0.54
Partial AIC = 571.02
log-likelihood ratio test = 2.90 on 6 df
-log2(p) of ll-ratio test = 0.28

(146, 25)


<lifelines.CoxPHFitter: fitted with 120 total observations, 50 right-censored observations>
             duration col = 'PFS.time'
                event col = 'PFS'
      baseline estimation = breslow
   number of observations = 120
number of events observed = 70
   partial log-likelihood = -279.51
         time fit was run = 2023-02-03 22:57:51 UTC

---
              coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                           
Age          -0.00       1.00       0.01            -0.03             0.02                 0.97                 1.02
bcell         0.08       1.08       0.20            -0.32             0.47                 0.73                 1.60
hpv          -0.19       0.83       0.47            -1.11             0.73                 0.33                 2.08
Sex           0.05       1.06       0.31            -0.55             0.66                 0.58                 1.93
II_thresh20  -0.00       1.00       0.01            -0.01             0.01                 0.99                 1.01
stage         0.19       1.21       0.15            -0.09             0.48                 0.91                 1.61

                z    p   -log2(p)
covariate                        
Age         -0.24 0.81       0.30
bcell        0.38 0.70       0.51
hpv         -0.40 0.69       0.53
Sex          0.17 0.86       0.21
II_thresh20 -0.14 0.89       0.18
stage        1.34 0.18       2.46
---
Concordance = 0.54
Partial AIC = 571.01
log-likelihood ratio test = 2.91 on 6 df
-log2(p) of ll-ratio test = 0.29

(146, 25)


<lifelines.CoxPHFitter: fitted with 120 total observations, 50 right-censored observations>
             duration col = 'PFS.time'
                event col = 'PFS'
      baseline estimation = breslow
   number of observations = 120
number of events observed = 70
   partial log-likelihood = -279.42
         time fit was run = 2023-02-03 22:58:00 UTC

---
              coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                           
Age          -0.00       1.00       0.01            -0.03             0.02                 0.97                 1.02
bcell         0.08       1.08       0.20            -0.31             0.47                 0.73                 1.60
hpv          -0.18       0.83       0.47            -1.10             0.74                 0.33                 2.09
Sex           0.05       1.05       0.31            -0.55             0.65                 0.58                 1.92
I_thresh0.5  -0.01       0.99       0.02            -0.04             0.03                 0.96                 1.03
stage         0.19       1.21       0.15            -0.09             0.48                 0.91                 1.61

                z    p   -log2(p)
covariate                        
Age         -0.20 0.84       0.24
bcell        0.40 0.69       0.53
hpv         -0.39 0.70       0.52
Sex          0.16 0.87       0.20
I_thresh0.5 -0.43 0.67       0.58
stage        1.34 0.18       2.47
---
Concordance = 0.54
Partial AIC = 570.84
log-likelihood ratio test = 3.08 on 6 df
-log2(p) of ll-ratio test = 0.32

(146, 25)


<lifelines.CoxPHFitter: fitted with 120 total observations, 50 right-censored observations>
             duration col = 'PFS.time'
                event col = 'PFS'
      baseline estimation = breslow
   number of observations = 120
number of events observed = 70
   partial log-likelihood = -279.47
         time fit was run = 2023-02-03 22:58:09 UTC

---
              coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                           
Age          -0.00       1.00       0.01            -0.03             0.02                 0.97                 1.02
bcell         0.08       1.08       0.20            -0.32             0.47                 0.73                 1.60
hpv          -0.18       0.83       0.47            -1.10             0.74                 0.33                 2.09
Sex           0.05       1.05       0.31            -0.55             0.65                 0.57                 1.92
II_thresh10  -0.00       1.00       0.01            -0.03             0.02                 0.97                 1.02
stage         0.20       1.22       0.15            -0.09             0.48                 0.91                 1.62

                z    p   -log2(p)
covariate                        
Age         -0.22 0.83       0.27
bcell        0.38 0.70       0.51
hpv         -0.39 0.70       0.52
Sex          0.16 0.87       0.20
II_thresh10 -0.31 0.75       0.41
stage        1.35 0.18       2.50
---
Concordance = 0.54
Partial AIC = 570.93
log-likelihood ratio test = 2.99 on 6 df
-log2(p) of ll-ratio test = 0.30

In [6]:
def cox_p_analysis_tert_high_neoantigen(df,antigen,include_stage = False,plot = False, save = False):
    
    from lifelines import CoxPHFitter
    cph = CoxPHFitter()

    """@DIR is a global variable"""
    f_dir = "C:/Data/Lab/Zanetti_Lab/HNSC/"
    pfs = pd.read_csv(f_dir + "PFSfile.csv")
    clinic = pd.read_csv(f_dir + "clinical.csv")
    clinic.rename(columns = {"Unnamed: 0":"patient_id"},inplace = True)
    clinic["patient_id"] = [ x.split("-")[2] for x in clinic["patient_id"]]
    
    # get log Tumor mutation burden
    allmaf = pd.read_csv(f_dir + "all.maf",header = None, sep = "\t")
    tmb = pd.DataFrame(allmaf[0].value_counts())
    tmb.index = [ x.split("-")[2] for x in tmb.index.tolist()]
    tmb.reset_index(inplace = True)
    tmb.columns = ["patient_id","mutCount"]
    tmb["logTMB"] = np.log(1 + tmb["mutCount"])
    
    df = pd.merge(df,pfs[["patient_id","PFS.time","PFS"]], on = "patient_id")
    df = pd.merge(df,clinic[["patient_id","Sex","Age"]], on = "patient_id")
    df = pd.merge(df,tmb[["patient_id","logTMB"]], on = "patient_id")
    df["Sex"] = [1 if x == "Male" else 0 for x in df.Sex.tolist()]
    df.dropna(inplace = True)
    df.drop_duplicates(inplace = True)
    df.dropna()
    
    df = pd.merge(df,neoantigen_data,on = "patient_id")
    df["hpv"] = [1 if x in hpv_pos_ids_cbioportal else 0 for x in df["patient_id"].tolist()]
    high_quantile = np.quantile(df["TERT"].tolist(),q = .7)
    df = df[df["TERT"] >= high_quantile].drop_duplicates()
    print(df.shape)
    
    # if include stage:

    if include_stage:
        stage_dict = {"Stage I":1,"Stage II":2, "Stage III":3, "Stage IV":4}
        stage = pd.read_csv(f_dir + "stageinfo.csv",header = None)
        stage.columns = ["patient_id","stage"]
        df = pd.merge(df, stage, on = "patient_id").drop_duplicates()
        df["stage"] = [stage_dict[x] if x in stage_dict.keys() else np.nan for x in df["stage"].tolist()]
        cph.fit(df[["PFS.time","PFS","bcell","hpv","Age","Sex",antigen,"stage"]].dropna(), 'PFS.time', event_col='PFS') # "bcell"
        cph.print_summary()
    else:
        cph.fit(df[[antigen,"PFS.time","PFS","Age","Sex"]], 'PFS.time', event_col='PFS')
        cph.print_summary()
        
    if plot:
        save_dir = "C:/Data/Lab/Zanetti_Lab/Figures/"
        cph.plot()
        if save:
            if include_stage:
                plt.savefig(save_dir+".cph.stage.plot.pdf", bbox_inches = "tight")
            else:
                plt.savefig(save_dir+".cph.plot.pdf", bbox_inches = "tight")

for antigen in ["I_thresh2","II_thresh20","I_thresh0.5","II_thresh10"]:
    cox_p_analysis_tert_high_neoantigen(df,antigen,include_stage = True)

(146, 25)


<lifelines.CoxPHFitter: fitted with 105 total observations, 60 right-censored observations>
             duration col = 'PFS.time'
                event col = 'PFS'
      baseline estimation = breslow
   number of observations = 105
number of events observed = 45
   partial log-likelihood = -172.50
         time fit was run = 2023-02-03 22:58:19 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
bcell      -0.42       0.66       0.19            -0.79            -0.04                 0.46                 0.96
hpv        -0.32       0.73       0.46            -1.22             0.59                 0.29                 1.80
Age         0.02       1.02       0.01            -0.01             0.04                 0.99                 1.05
Sex        -0.18       0.83       0.34            -0.85             0.48                 0.43                 1.62
I_thresh2  -0.01       0.99       0.01            -0.04             0.01                 0.96                 1.01
stage       0.28       1.33       0.17            -0.05             0.62                 0.95                 1.86

              z    p   -log2(p)
covariate                      
bcell     -2.20 0.03       5.16
hpv       -0.69 0.49       1.02
Age        1.17 0.24       2.04
Sex       -0.54 0.59       0.77
I_thresh2 -1.03 0.30       1.72
stage      1.64 0.10       3.32
---
Concordance = 0.67
Partial AIC = 356.99
log-likelihood ratio test = 14.99 on 6 df
-log2(p) of ll-ratio test = 5.62

(146, 25)


<lifelines.CoxPHFitter: fitted with 105 total observations, 60 right-censored observations>
             duration col = 'PFS.time'
                event col = 'PFS'
      baseline estimation = breslow
   number of observations = 105
number of events observed = 45
   partial log-likelihood = -172.48
         time fit was run = 2023-02-03 22:58:30 UTC

---
              coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                           
bcell        -0.41       0.66       0.19            -0.78            -0.04                 0.46                 0.96
hpv          -0.27       0.76       0.46            -1.18             0.63                 0.31                 1.88
Age           0.02       1.02       0.01            -0.01             0.04                 0.99                 1.04
Sex          -0.16       0.85       0.34            -0.83             0.51                 0.43                 1.67
II_thresh20  -0.01       0.99       0.01            -0.04             0.01                 0.96                 1.01
stage         0.29       1.34       0.17            -0.05             0.64                 0.95                 1.89

                z    p   -log2(p)
covariate                        
bcell       -2.17 0.03       5.05
hpv         -0.59 0.56       0.85
Age          1.15 0.25       2.01
Sex         -0.47 0.64       0.64
II_thresh20 -1.04 0.30       1.76
stage        1.69 0.09       3.45
---
Concordance = 0.67
Partial AIC = 356.96
log-likelihood ratio test = 15.02 on 6 df
-log2(p) of ll-ratio test = 5.64

(146, 25)


<lifelines.CoxPHFitter: fitted with 105 total observations, 60 right-censored observations>
             duration col = 'PFS.time'
                event col = 'PFS'
      baseline estimation = breslow
   number of observations = 105
number of events observed = 45
   partial log-likelihood = -172.15
         time fit was run = 2023-02-03 22:58:39 UTC

---
              coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                           
bcell        -0.44       0.65       0.19            -0.81            -0.06                 0.44                 0.94
hpv          -0.29       0.75       0.46            -1.19             0.62                 0.30                 1.85
Age           0.02       1.02       0.01            -0.01             0.05                 0.99                 1.05
Sex          -0.20       0.82       0.34            -0.86             0.46                 0.42                 1.59
I_thresh0.5  -0.05       0.95       0.04            -0.12             0.03                 0.89                 1.03
stage         0.29       1.34       0.17            -0.05             0.63                 0.96                 1.88

                z    p   -log2(p)
covariate                        
bcell       -2.27 0.02       5.44
hpv         -0.62 0.53       0.91
Age          1.26 0.21       2.27
Sex         -0.58 0.56       0.84
I_thresh0.5 -1.27 0.20       2.29
stage        1.70 0.09       3.47
---
Concordance = 0.66
Partial AIC = 356.31
log-likelihood ratio test = 15.67 on 6 df
-log2(p) of ll-ratio test = 6.00

(146, 25)


<lifelines.CoxPHFitter: fitted with 105 total observations, 60 right-censored observations>
             duration col = 'PFS.time'
                event col = 'PFS'
      baseline estimation = breslow
   number of observations = 105
number of events observed = 45
   partial log-likelihood = -172.35
         time fit was run = 2023-02-03 22:58:47 UTC

---
              coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                           
bcell        -0.42       0.66       0.19            -0.79            -0.04                 0.45                 0.96
hpv          -0.29       0.74       0.46            -1.20             0.61                 0.30                 1.85
Age           0.02       1.02       0.01            -0.01             0.04                 0.99                 1.04
Sex          -0.15       0.86       0.34            -0.82             0.53                 0.44                 1.70
II_thresh10  -0.02       0.98       0.02            -0.06             0.02                 0.94                 1.02
stage         0.30       1.35       0.17            -0.04             0.64                 0.96                 1.90

                z    p   -log2(p)
covariate                        
bcell       -2.19 0.03       5.13
hpv         -0.64 0.52       0.93
Age          1.17 0.24       2.05
Sex         -0.42 0.67       0.58
II_thresh10 -1.16 0.25       2.03
stage        1.72 0.09       3.55
---
Concordance = 0.67
Partial AIC = 356.70
log-likelihood ratio test = 15.28 on 6 df
-log2(p) of ll-ratio test = 5.78

# Table S2. other conserved antigen

In [7]:
def cox_p_analysis_other_antigen(df,antigen,include_stage = False,plot = False, save = False):
    
    from lifelines import CoxPHFitter
    cph = CoxPHFitter()

    """@DIR is a global variable"""
    f_dir = "C:/Data/Lab/Zanetti_Lab/HNSC/"
    pfs = pd.read_csv(f_dir + "PFSfile.csv")
    clinic = pd.read_csv(f_dir + "clinical.csv")
    clinic.rename(columns = {"Unnamed: 0":"patient_id"},inplace = True)
    clinic["patient_id"] = [ x.split("-")[2] for x in clinic["patient_id"]]
    
    # get log Tumor mutation burden
    allmaf = pd.read_csv(f_dir + "all.maf",header = None, sep = "\t")
    tmb = pd.DataFrame(allmaf[0].value_counts())
    tmb.index = [ x.split("-")[2] for x in tmb.index.tolist()]
    tmb.reset_index(inplace = True)
    tmb.columns = ["patient_id","mutCount"]
    tmb["logTMB"] = np.log(1 + tmb["mutCount"])
    
    df = pd.merge(df,pfs[["patient_id","PFS.time","PFS"]], on = "patient_id")
    df = pd.merge(df,clinic[["patient_id","Sex","Age"]], on = "patient_id")
    df = pd.merge(df,tmb[["patient_id","logTMB"]], on = "patient_id")
    df["Sex"] = [1 if x == "Male" else 0 for x in df.Sex.tolist()]
    df.dropna(inplace = True)
    df.drop_duplicates(inplace = True)
    df.dropna()
    
    df["hpv"] = [1 if x in hpv_pos_ids_cbioportal else 0 for x in df["patient_id"].tolist()]
    low_quantile = np.quantile(df["TERT"].tolist(),q = .3)
    df = df[df["TERT"] <= low_quantile]
    print(df.shape)
    
    # if include stage:

    if include_stage:
        stage_dict = {"Stage I":1,"Stage II":2, "Stage III":3, "Stage IV":4}
        stage = pd.read_csv(f_dir + "stageinfo.csv",header = None)
        stage.columns = ["patient_id","stage"]
        df = pd.merge(df, stage, on = "patient_id").drop_duplicates()
        df["stage"] = [stage_dict[x] if x in stage_dict.keys() else np.nan for x in df["stage"].tolist()]
        cph.fit(df[["PFS.time","PFS","Age","Sex",antigen,"logTMB","stage","hpv"]].dropna(), 'PFS.time', event_col='PFS')
        cph.print_summary()
    else:
        cph.fit(df[["logTMB",antigen,"PFS.time","PFS","Age","Sex","hpv"]], 'PFS.time', event_col='PFS')
        cph.print_summary()
        
    if plot:
        save_dir = "C:/Data/Lab/Zanetti_Lab/Figures/"
        cph.plot()
        if save:
            if include_stage:
                plt.savefig(save_dir+".cph.stage.plot.pdf", bbox_inches = "tight")
            else:
                plt.savefig(save_dir+".cph.plot.pdf", bbox_inches = "tight")

#"MAGEA4","MAGEA3","CEACAM5"
for antigen in ["CTAG1B","MUC1","MAGEA4","MAGEA3","CEACAM5"]:
    cox_p_analysis_other_antigen(hnsc,antigen,include_stage = True, plot = False)

(150, 27115)


<lifelines.CoxPHFitter: fitted with 122 total observations, 52 right-censored observations>
             duration col = 'PFS.time'
                event col = 'PFS'
      baseline estimation = breslow
   number of observations = 122
number of events observed = 70
   partial log-likelihood = -278.61
         time fit was run = 2023-02-03 22:59:11 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
Age        -0.00       1.00       0.01            -0.03             0.02                 0.97                 1.02
Sex         0.06       1.06       0.32            -0.56             0.68                 0.57                 1.97
CTAG1B     -1.02       0.36       1.05            -3.08             1.03                 0.05                 2.80
logTMB      0.12       1.13       0.17            -0.22             0.46                 0.80                 1.58
stage       0.23       1.26       0.15            -0.05             0.52                 0.95                 1.68
hpv        -0.11       0.89       0.47            -1.04             0.81                 0.35                 2.26

              z    p   -log2(p)
covariate                      
Age       -0.28 0.78       0.36
Sex        0.19 0.85       0.24
CTAG1B    -0.98 0.33       1.61
logTMB     0.68 0.50       1.01
stage      1.61 0.11       3.21
hpv       -0.24 0.81       0.31
---
Concordance = 0.59
Partial AIC = 569.22
log-likelihood ratio test = 9.03 on 6 df
-log2(p) of ll-ratio test = 2.54

(150, 27115)


<lifelines.CoxPHFitter: fitted with 122 total observations, 52 right-censored observations>
             duration col = 'PFS.time'
                event col = 'PFS'
      baseline estimation = breslow
   number of observations = 122
number of events observed = 70
   partial log-likelihood = -280.61
         time fit was run = 2023-02-03 22:59:33 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
Age        -0.00       1.00       0.01            -0.03             0.02                 0.97                 1.02
Sex         0.07       1.07       0.32            -0.55             0.69                 0.57                 1.98
MUC1        0.00       1.00       0.00            -0.00             0.01                 1.00                 1.01
logTMB      0.03       1.03       0.17            -0.29             0.36                 0.75                 1.43
stage       0.25       1.28       0.15            -0.04             0.53                 0.96                 1.71
hpv        -0.28       0.76       0.49            -1.23             0.67                 0.29                 1.96

              z    p   -log2(p)
covariate                      
Age       -0.27 0.78       0.35
Sex        0.21 0.84       0.26
MUC1       1.28 0.20       2.32
logTMB     0.20 0.84       0.25
stage      1.69 0.09       3.46
hpv       -0.58 0.56       0.83
---
Concordance = 0.56
Partial AIC = 573.21
log-likelihood ratio test = 5.04 on 6 df
-log2(p) of ll-ratio test = 0.89

(150, 27115)


<lifelines.CoxPHFitter: fitted with 122 total observations, 52 right-censored observations>
             duration col = 'PFS.time'
                event col = 'PFS'
      baseline estimation = breslow
   number of observations = 122
number of events observed = 70
   partial log-likelihood = -280.70
         time fit was run = 2023-02-03 22:59:54 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
Age        -0.00       1.00       0.01            -0.03             0.02                 0.98                 1.02
Sex         0.10       1.10       0.32            -0.52             0.72                 0.60                 2.05
MAGEA4     -0.00       1.00       0.00            -0.01             0.00                 0.99                 1.00
logTMB      0.09       1.10       0.17            -0.24             0.43                 0.78                 1.54
stage       0.22       1.25       0.14            -0.06             0.50                 0.94                 1.66
hpv        -0.16       0.85       0.47            -1.08             0.76                 0.34                 2.14

              z    p   -log2(p)
covariate                      
Age       -0.06 0.95       0.08
Sex        0.31 0.75       0.41
MAGEA4    -1.06 0.29       1.80
logTMB     0.55 0.58       0.78
stage      1.52 0.13       2.96
hpv       -0.34 0.73       0.45
---
Concordance = 0.56
Partial AIC = 573.40
log-likelihood ratio test = 4.85 on 6 df
-log2(p) of ll-ratio test = 0.83

(150, 27115)


<lifelines.CoxPHFitter: fitted with 122 total observations, 52 right-censored observations>
             duration col = 'PFS.time'
                event col = 'PFS'
      baseline estimation = breslow
   number of observations = 122
number of events observed = 70
   partial log-likelihood = -278.88
         time fit was run = 2023-02-03 23:00:15 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
Age         0.00       1.00       0.01            -0.02             0.03                 0.98                 1.03
Sex         0.12       1.12       0.32            -0.51             0.74                 0.60                 2.10
MAGEA3     -0.01       0.99       0.00            -0.02            -0.00                 0.98                 1.00
logTMB      0.10       1.10       0.17            -0.23             0.43                 0.79                 1.53
stage       0.23       1.26       0.15            -0.06             0.52                 0.95                 1.68
hpv        -0.27       0.77       0.47            -1.19             0.66                 0.30                 1.93

              z    p   -log2(p)
covariate                      
Age        0.07 0.95       0.08
Sex        0.37 0.71       0.49
MAGEA3    -2.00 0.05       4.46
logTMB     0.58 0.56       0.84
stage      1.58 0.11       3.14
hpv       -0.56 0.57       0.80
---
Concordance = 0.59
Partial AIC = 569.77
log-likelihood ratio test = 8.48 on 6 df
-log2(p) of ll-ratio test = 2.29

(150, 27115)


<lifelines.CoxPHFitter: fitted with 122 total observations, 52 right-censored observations>
             duration col = 'PFS.time'
                event col = 'PFS'
      baseline estimation = breslow
   number of observations = 122
number of events observed = 70
   partial log-likelihood = -280.85
         time fit was run = 2023-02-03 23:00:36 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
Age        -0.00       1.00       0.01            -0.03             0.02                 0.98                 1.02
Sex         0.11       1.12       0.31            -0.50             0.73                 0.61                 2.07
CEACAM5    -0.00       1.00       0.00            -0.00             0.00                 1.00                 1.00
logTMB      0.07       1.08       0.17            -0.26             0.41                 0.77                 1.50
stage       0.20       1.22       0.15            -0.09             0.49                 0.92                 1.63
hpv         0.02       1.02       0.49            -0.94             0.99                 0.39                 2.70

              z    p   -log2(p)
covariate                      
Age       -0.05 0.96       0.05
Sex        0.37 0.71       0.49
CEACAM5   -0.92 0.36       1.49
logTMB     0.44 0.66       0.60
stage      1.36 0.17       2.52
hpv        0.05 0.96       0.06
---
Concordance = 0.59
Partial AIC = 573.70
log-likelihood ratio test = 4.55 on 6 df
-log2(p) of ll-ratio test = 0.73

# Table S3. other conserved antigen¶

In [8]:
cibersortx_hnsc = pd.read_csv("C:/Data/Lab/Zanetti_Lab/HNSC/PNASNEXUS/CIBERSORTxGEP_Job3_GEPs_Filtered.txt",sep = "\t")
cibersortx_hnsc.set_index("GeneSymbol").loc["TERT"]

B cells            0.000000
Plasma cells       0.693851
T cells CD8        0.144039
T cells CD4        1.539002
NK cells           0.000000
Monocytes          0.000000
Dendritic cells    0.000000
Mast cells         0.015191
Eosinophils        0.000000
Neutrophils        0.000000
Name: TERT, dtype: float64